In [1]:
import SimpleITK as sitk
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import shutil
import numpy as np
from scipy.spatial.transform import Rotation as R


In [2]:
def get_metadata_maybe(key, sitk_img, default='not_found'):
    try:
        value = sitk_img.GetMetaData(key)
    except Exception as e:
        #logging.debug('key not found: {}, {}'.format(key, e))
        value = default
        pass

    return value

In [3]:
def show_3D(sitk_img):
    img_array = sitk.GetArrayFromImage(sitk_img)
    max_depth = img_array.shape[0]
    fig = plt.figure(figsize=(30,max_depth))
    for i in range(max_depth):
        fig.add_subplot(1,max_depth,i+1)
        plt.imshow(img_array[i,:,:])
        plt.axis('off')

In [4]:
def Image3D_from_Time(sitk_img,time):
    img_array = sitk.GetArrayFromImage(sitk_img)
    sitk_array_3D = img_array[time-1,:,:,:] 
    new_sitk_img = sitk.GetImageFromArray(sitk_array_3D)
    return new_sitk_img

In [5]:
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [6]:
#returns img of only one label
def get_single_label_img(sitk_img, label):
    img_array = sitk.GetArrayFromImage(sitk_img)
    label_array = (img_array == label).astype(int)
    label_array = label_array * 100
    label_img = sitk.GetImageFromArray(label_array)
    label_img.CopyInformation(sitk_img)
    return label_img

In [7]:
#returns resampled img1 by referncing img2
def resample_img(sitk_img1, sitk_img2):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(sitk_img2)
    resampled_img = resampler.Execute(sitk_img1)
    return resampled_img

In [8]:
#returns resampled_img back to normal label
def convert_back_to_label(sitk_img, label):
    img_array = sitk.GetArrayFromImage(sitk_img)
    label_array = (img_array > 25).astype(int)
    label_array = label_array * label
    label_img = sitk.GetImageFromArray(label_array)
    label_img.CopyInformation(sitk_img)
    return label_img

In [9]:
#returns resampled_img back to normal label with percentage
def advanced_convert_back_to_label(sitk_img, label, percentage):
    img_array = sitk.GetArrayFromImage(sitk_img)
    percent_array = img_array.flatten()
    percent_array = percent_array[percent_array > 0]
    try:
        threshold = - np.percentile(percent_array*-1,q=percentage)
    except IndexError:
        print("IndexError")
        threshold = 50
    label_array = (img_array >= threshold).astype(int)
    label_array = label_array * label
    label_img = sitk.GetImageFromArray(label_array)
    label_img.CopyInformation(sitk_img)
    return label_img

In [10]:
#returns all labels added and with priority from label 3 to 1
def add_labels(label_array1,label_array2,label_array3):
    final_array = label_array3
    temp_array = final_array + label_array2
    #if you add the label, there is only a 2 if there was a 0 there before, so now oferwriting
    temp_array = (temp_array == 2).astype(int)
    temp_array = temp_array * 2
    final_array = final_array + temp_array
    temp_array = final_array + label_array1
    temp_array = (temp_array == 1).astype(int)
    temp_array = temp_array * 1
    final_array = final_array + temp_array
    return final_array

In [11]:
def add_labels_max_thres(label_array1, label_array2, label_array3, threshold):
    max_label_1_2_array = np.maximum(label_array1, label_array2)
    max_label_2_3_array = np.maximum(label_array2, label_array3)
    max_label_array = np.maximum(max_label_1_2_array, max_label_2_3_array)    
    thres_label_array = (max_label_array >= threshold).astype(int)    
    max_label_array = thres_label_array * max_label_array   
    
    max_label_array1 =  label_array1 * np.equal(max_label_array, label_array1).astype(int)
    #max_label_array1 =  (max_label_array1 > threshold).astype(int) * 1
    max_label_array1 =  max_label_array1.astype(bool).astype(int) * 1
    max_label_array2 =  label_array2 * np.equal(max_label_array, label_array2).astype(int)
    #max_label_array2 =  (max_label_array2 > threshold).astype(int) * 2
    max_label_array2 =  max_label_array2.astype(bool).astype(int) * 2
    max_label_array3 =  label_array3 * np.equal(max_label_array, label_array3).astype(int)
    #max_label_array3 =  (max_label_array3 > threshold).astype(int) * 3
    max_label_array3 =  max_label_array3.astype(bool).astype(int) * 3
    
    return add_labels(max_label_array1, max_label_array2, max_label_array3)    

In [12]:
#reutrns resampled img1 by referencing img2
def resample_label_img(sitk_img1, sitk_img2):
    label1_img1 = get_single_label_img(sitk_img1,1)
    label2_img1 = get_single_label_img(sitk_img1,2)
    label3_img1 = get_single_label_img(sitk_img1,3)
    
    resampled_label1 = resample_img(label1_img1, sitk_img2)
    resampled_label2 = resample_img(label2_img1, sitk_img2)
    resampled_label3 = resample_img(label3_img1, sitk_img2)
    resampled_label1 = convert_back_to_label(resampled_label1,1)
    resampled_label2 = convert_back_to_label(resampled_label2,2)
    resampled_label3 = convert_back_to_label(resampled_label3,3)
    
    resampled_array1 = sitk.GetArrayFromImage(resampled_label1)
    resampled_array2 = sitk.GetArrayFromImage(resampled_label2)
    resampled_array3 = sitk.GetArrayFromImage(resampled_label3)
    resampled_array = add_labels(resampled_array1, resampled_array2, resampled_array3)
    resampled_img= sitk.GetImageFromArray(resampled_array)
    resampled_img.CopyInformation(resampled_label1)
    return resampled_img

In [13]:
#reutrns resampled img1 by referencing img2 with percentage
def percentage_resample_label_img(sitk_img1, sitk_img2, percentage):
    label1_img1 = get_single_label_img(sitk_img1,1)
    label2_img1 = get_single_label_img(sitk_img1,2)
    label3_img1 = get_single_label_img(sitk_img1,3)
    
    resampled_label1 = resample_img(label1_img1, sitk_img2)
    resampled_label2 = resample_img(label2_img1, sitk_img2)
    resampled_label3 = resample_img(label3_img1, sitk_img2)
    resampled_label1 = advanced_convert_back_to_label(resampled_label1,1, percentage)
    resampled_label2 = advanced_convert_back_to_label(resampled_label2,2, percentage)
    resampled_label3 = advanced_convert_back_to_label(resampled_label3,3, percentage)
    
    resampled_array1 = sitk.GetArrayFromImage(resampled_label1)
    resampled_array2 = sitk.GetArrayFromImage(resampled_label2)
    resampled_array3 = sitk.GetArrayFromImage(resampled_label3)
    resampled_array = add_labels(resampled_array1, resampled_array2, resampled_array3)
    resampled_img= sitk.GetImageFromArray(resampled_array)
    resampled_img.CopyInformation(resampled_label1)
    return resampled_img

In [14]:
#reutrns resampled img1 by referencing img2 with percentage
def max_thres_resample_label_img(sitk_img1, sitk_img2, threshold):
    label1_img1 = get_single_label_img(sitk_img1,1)
    label2_img1 = get_single_label_img(sitk_img1,2)
    label3_img1 = get_single_label_img(sitk_img1,3)
    
    resampled_label1 = resample_img(label1_img1, sitk_img2)
    resampled_label2 = resample_img(label2_img1, sitk_img2)
    resampled_label3 = resample_img(label3_img1, sitk_img2) 
    
    resampled_array1 = sitk.GetArrayFromImage(resampled_label1)
    resampled_array2 = sitk.GetArrayFromImage(resampled_label2)
    resampled_array3 = sitk.GetArrayFromImage(resampled_label3)
    
    resampled_array = add_labels_max_thres(resampled_array1, resampled_array2, resampled_array3, threshold)
    resampled_img= sitk.GetImageFromArray(resampled_array)
    resampled_img.CopyInformation(resampled_label1)
    return resampled_img

In [15]:
#returns resampled img1 by referencing img2 with percentage.
def max_thres_resample2_label_img(sitk_img1, sitk_img2, threshold):
    label1_img1 = get_single_label_img(sitk_img1,1)
    label2_img1 = get_single_label_img(sitk_img1,2)
    label3_img1 = get_single_label_img(sitk_img1,3)
    
    resampled_label1 = resample_direcion_origin_spacing(label1_img1, sitk_img2, interpolate=sitk.sitkLinear)
    resampled_label2 = resample_direcion_origin_spacing(label2_img1, sitk_img2, interpolate=sitk.sitkLinear)
    resampled_label3 = resample_direcion_origin_spacing(label3_img1, sitk_img2, interpolate=sitk.sitkLinear) 
    
    resampled_array1 = sitk.GetArrayFromImage(resampled_label1)
    resampled_array2 = sitk.GetArrayFromImage(resampled_label2)
    resampled_array3 = sitk.GetArrayFromImage(resampled_label3)
    
    
    resampled_array = add_labels_max_thres(resampled_array1, resampled_array2, resampled_array3, threshold)
    resampled_img= sitk.GetImageFromArray(resampled_array)
    resampled_img = sitk.Cast(resampled_img, sitk.sitkUInt8)
    resampled_img.CopyInformation(resampled_label1)
    return resampled_img

In [16]:
def resample_direcion_origin_spacing(sitk_img, reference_sitk, interpolate=sitk.sitkLinear):
    """
    Resample a sitk img, copy direction, origin and spacing of the reference image
    Keep the size (resolution) of the target image
    :param sitk_img: sitk.Image
    :param reference_sitk: sitk.Image
    :param interpolate: fn
    :return: 
    """
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(reference_sitk.GetSpacing())
    resampler.SetInterpolator(interpolate)
    resampler.SetOutputDirection(reference_sitk.GetDirection())
    resampler.SetOutputOrigin(reference_sitk.GetOrigin())
    resampler.SetSize(reference_sitk.GetSize())
    resampled = resampler.Execute(sitk_img)
    # copy metadata
    for key in reference_sitk.GetMetaDataKeys():
        resampled.SetMetaData(key, get_metadata_maybe(reference_sitk, key))
    return resampled

In [17]:
#reutrns resampled img1 by referencing img2 with percentage, modified by sven !!!
def max_thres_resample2_iso_label_img(sitk_img1, threshold, spacing_=(1.5,1.5,1.5),file_path='temp.nrrd', interpol=sitk.sitkLinear):
    from src.visualization.Visualize import show_2D_or_3D
    from src.data.Postprocess import clean_3d_prediction_3d_cc
    import numpy as np
    import cv2
    import scipy
    debug=False
    
    label1_img1 = get_single_label_img(sitk_img1,1)
    label2_img1 = get_single_label_img(sitk_img1,2)
    label3_img1 = get_single_label_img(sitk_img1,3)
    
    resampled_label1, _ = transform_to_isotrop_voxels(label1_img1, interpolate=interpol, spacing_=spacing_)
    resampled_label2, _ = transform_to_isotrop_voxels(label2_img1, interpolate=interpol, spacing_=spacing_)
    resampled_label3, _ = transform_to_isotrop_voxels(label3_img1, interpolate=interpol, spacing_=spacing_)
    
    resampled_array1 = sitk.GetArrayFromImage(resampled_label1)
    resampled_array2 = sitk.GetArrayFromImage(resampled_label2)
    resampled_array3 = sitk.GetArrayFromImage(resampled_label3)
    
    kernel = np.ones((5,5,7),np.uint8)
    kernel_small = np.ones((3,3,3),np.uint8)
    # maybe use a bigger/smaller kernel?
    
    # close holes, smooth in z,x,y
    resampled_array1 = scipy.ndimage.morphology.grey_closing(resampled_array1,structure=kernel, mode='constant')
    resampled_array2 = scipy.ndimage.morphology.grey_closing(resampled_array2,structure=kernel, mode='constant')
    resampled_array3 = scipy.ndimage.morphology.grey_closing(resampled_array3,structure=kernel, mode='constant')
    
    resampled_array1 = scipy.ndimage.morphology.grey_closing(resampled_array1,structure=kernel_small, mode='constant')
    resampled_array2 = scipy.ndimage.morphology.grey_closing(resampled_array2,structure=kernel_small, mode='constant')
    resampled_array3 = scipy.ndimage.morphology.grey_closing(resampled_array3,structure=kernel_small, mode='constant')
    
    
    if debug: show_2D_or_3D(resampled_array1[::10])
    plt.show()
    if debug: show_2D_or_3D(resampled_array2[::10])
    plt.show()
    if debug: show_2D_or_3D(resampled_array3[::10])
    plt.show()
    
    resampled_array = add_labels_max_thres(resampled_array1, resampled_array2, resampled_array3, threshold)
    

    
    resampled_img= sitk.GetImageFromArray(resampled_array)
    resampled_img = sitk.Cast(resampled_img, sitk.sitkUInt8)
    resampled_img.CopyInformation(resampled_label1)
    return resampled_img, file_path

In [18]:
#reutrns resampled img1 by referencing img2 with percentage, original from the TMI paper!
def max_thres_resample2_iso_label_img_original(sitk_img1, threshold):
    from src.visualization.Visualize import show_2D_or_3D
    debug=True
    
    label1_img1 = get_single_label_img(sitk_img1,1)
    label2_img1 = get_single_label_img(sitk_img1,2)
    label3_img1 = get_single_label_img(sitk_img1,3)
    
    resampled_label1 = transform_to_isotrop_voxels(label1_img1, interpolate=sitk.sitkLinear, spacing_=(1.5,1.5,1.5))
    resampled_label2 = transform_to_isotrop_voxels(label2_img1, interpolate=sitk.sitkLinear, spacing_=(1.5,1.5,1.5))
    resampled_label3 = transform_to_isotrop_voxels(label3_img1, interpolate=sitk.sitkLinear, spacing_=(1.5,1.5,1.5))
    
    resampled_array1 = sitk.GetArrayFromImage(resampled_label1)
    resampled_array2 = sitk.GetArrayFromImage(resampled_label2)
    resampled_array3 = sitk.GetArrayFromImage(resampled_label3)

    if debug: show_2D_or_3D(resampled_array1[::10])
    plt.show()
    if debug: show_2D_or_3D(resampled_array2[::10])
    plt.show()
    if debug: show_2D_or_3D(resampled_array3[::10])
    plt.show()
    
    resampled_array = add_labels_max_thres(resampled_array1, resampled_array2, resampled_array3, threshold)
    resampled_img= sitk.GetImageFromArray(resampled_array)
    resampled_img = sitk.Cast(resampled_img, sitk.sitkUInt8)
    resampled_img.CopyInformation(resampled_label1)
    return resampled_img

In [19]:
#origin shift with transformindex
def resample_direcion_origin_spacing_shift(sitk_img, reference_sitk, shift, interpolate=sitk.sitkLinear, file_path='temp.nrrd'):
    """
    Resample a sitk img, copy direction, origin and spacing of the reference image
    Keep the size (resolution) of the target image
    :param sitk_img: sitk.Image
    :param reference_sitk: sitk.Image
    :param interpolate: fn
    :return: falsch muss umgeschrieben werden
    """
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(reference_sitk.GetSpacing())
    resampler.SetInterpolator(interpolate)
    resampler.SetOutputDirection(reference_sitk.GetDirection())
    #if coupled with resampler.SetSize only shift origin if negativ
    resampler.SetOutputOrigin(reference_sitk.TransformIndexToPhysicalPoint(tuple([abs(x)*-1 for x in shift])))
    #resampler.SetSize(sitk_img.GetSize())
    resampler.SetSize(tuple(map(sum,zip(reference_sitk.GetSize(), tuple([abs(x)*2 for x in shift])))))
    #resampler.SetSize(tuple(map(sum,zip(sitk_img.GetSize(), tuple([abs(x) for x in shift])))))
    resampled = resampler.Execute(sitk_img)
    # copy metadata
    for key in reference_sitk.GetMetaDataKeys():
        resampled.SetMetaData(key, get_metadata_maybe(reference_sitk, key))
    return resampled, file_path


#reutrns resampled img1 by referencing img2 with percentage
def max_thres_resample2_label_img_shift(sitk_img1, sitk_img2, threshold, shift, file_path='temp.nrrd'):
    label1_img1 = get_single_label_img(sitk_img1,1)
    label2_img1 = get_single_label_img(sitk_img1,2)
    label3_img1 = get_single_label_img(sitk_img1,3)
    
    resampled_label1, _ = resample_direcion_origin_spacing_shift(label1_img1, sitk_img2, shift, interpolate=sitk.sitkLinear)
    resampled_label2, _ = resample_direcion_origin_spacing_shift(label2_img1, sitk_img2, shift, interpolate=sitk.sitkLinear)
    resampled_label3, _ = resample_direcion_origin_spacing_shift(label3_img1, sitk_img2, shift, interpolate=sitk.sitkLinear) 
    
    resampled_array1 = sitk.GetArrayFromImage(resampled_label1)
    resampled_array2 = sitk.GetArrayFromImage(resampled_label2)
    resampled_array3 = sitk.GetArrayFromImage(resampled_label3)
    
    resampled_array = add_labels_max_thres(resampled_array1, resampled_array2, resampled_array3, threshold)
    resampled_img= sitk.GetImageFromArray(resampled_array)
    resampled_img = sitk.Cast(resampled_img, sitk.sitkUInt8)
    resampled_img.CopyInformation(resampled_label1)
    return resampled_img, file_path

In [20]:
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.Notebook_imports import *

search for root_dir and set working directory
Working directory set to: /mnt/ssd/git/3d-mri-domain-adaption


In [21]:
ax_src = '/mnt/ssd/data/gcn/ax_sax_from_flo/ax3d_iso_linear/'
sax_src = '/mnt/ssd/data/gcn/ax_sax_from_flo/sax3d_iso_linear/'
dst_ax2sax = '/mnt/ssd/data/gcn/ax_sax_from_flo/sax2ax3d_iso/'
ax2sax_iso_dst = '/mnt/ssd/data/gcn/ax_sax_from_flo/ax2sax3d_iso_linear/'
sax2ax_iso_dst = '/mnt/ssd/data/gcn/ax_sax_from_flo/sax2ax3d_iso_linear/'

# Resample to other orientation

In [23]:
# resample 3D image / mask from axial view into sax view or vize versa, extend the transformed vol by 10 voxel along the target z axis
import concurrent.futures
from concurrent.futures import as_completed

workers = 16
futures_ax2sax = set()
futures_sax2ax = set()
ensure_dir(ax2sax_iso_dst)
ensure_dir(sax2ax_iso_dst)
# spawn multiple threads or processes
# transform the image of file_path with according to the direction of 
ax_files = sorted(glob.glob(ax_src+'*.nrrd', recursive = True))
sax_files = sorted(glob.glob(sax_src+'*.nrrd', recursive = True))
print('axial files: {}'.format(len(ax_files)))
print('sax files: {}'.format(len(sax_files)))
with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
    
    for ax_file_path, sax_file_path in zip(ax_files, sax_files):
        
        reader1 = sitk.ImageFileReader()
        reader1.SetFileName(ax_file_path)
        ax_img = reader1.Execute()
        
        reader2 = sitk.ImageFileReader()
        reader2.SetFileName(sax_file_path)
        sax_img = reader2.Execute()
    
        if 'img' in os.path.basename(ax_file_path):
            futures_ax2sax.add(executor.submit(resample_direcion_origin_spacing_shift, ax_img, sax_img, (0,0,-10), sitk.sitkLinear, ax_file_path))
            futures_sax2ax.add(executor.submit(resample_direcion_origin_spacing_shift, sax_img, ax_img, (0,0,-10), sitk.sitkLinear, sax_file_path))

        else:
            futures_ax2sax.add(executor.submit(max_thres_resample2_label_img_shift, ax_img, sax_img, 50, (0,0,-10), ax_file_path))
            futures_sax2ax.add(executor.submit(max_thres_resample2_label_img_shift, sax_img, ax_img, 50, (0,0,-10), sax_file_path))

writer = sitk.ImageFileWriter()
for i, future in enumerate(as_completed(futures_ax2sax)):
    try:
        res = future.result()
        resampled, file_path = res
        writer.SetFileName(os.path.join(ax2sax_iso_dst,os.path.basename(file_path)))
        writer.Execute(resampled) 

    except Exception as e:
        logging.error(
            'Exception {} in datagenerator with: image: {} or mask: {}'.format(str(e)))
        
for i, future in enumerate(as_completed(futures_sax2ax)):
    try:
        res = future.result()
        resampled, file_path = res
        writer.SetFileName(os.path.join(sax2ax_iso_dst,os.path.basename(file_path)))
        writer.Execute(resampled) 

    except Exception as e:
        logging.error(
            'Exception {} in datagenerator with: image: {} or mask: {}'.format(str(e)))

axial files: 324
sax files: 324


# Resample to isotrop

This method creates strange artefacts, so far I dont know how to solve

In [20]:
ax_src = '/mnt/ssd/data/gcn/ax_sax_from_flo/ax3d/'
sax_src = '/mnt/ssd/data/gcn/ax_sax_from_flo/sax3d/'
ax_iso_dst = '/mnt/ssd/data/gcn/ax_sax_from_flo/ax3d_iso_linear/'
sax_iso_dst = '/mnt/ssd/data/gcn/ax_sax_from_flo/sax3d_iso_linear/'

In [21]:
def transform_to_isotrop_voxels(sitk_img, interpolate=sitk.sitkBSpline, spacing_=(1.,1.,1.), file_path='temp.nrrd'):

    size = sitk_img.GetSize()
    spacing = sitk_img.GetSpacing()
    size_new = tuple([int((s*space_old)//space_new) for s,space_old,space_new in zip(size,spacing,spacing_)])

    # resample to isotrop voxels
    resampler = sitk.ResampleImageFilter()
    resampler.SetSize(size_new)
    resampler.SetOutputSpacing(spacing_)
    resampler.SetOutputOrigin(sitk_img.GetOrigin())
    resampler.SetOutputDirection(sitk_img.GetDirection())
    resampler.SetInterpolator(interpolate) 
    resampled = resampler.Execute(sitk_img)
        # copy metadata
    for key in sitk_img.GetMetaDataKeys():
        resampled.SetMetaData(key, get_metadata_maybe(sitk_img, key))
    return resampled, file_path

In [23]:
# resample 3D image / mask into a fake isotrop resolution, use multithreadding, combine iso and orientation resample, everything in one step
import concurrent.futures
from concurrent.futures import as_completed
slice_first = 2

workers = 16
futures_ax_iso = set()
futures_sax_iso = set()
futures_ax2sax_iso = set()
futures_sax2ax_iso = set()

ax_files = sorted(glob.glob(ax_src+'*.nrrd', recursive = True))
sax_files = sorted(glob.glob(sax_src+'*.nrrd', recursive = True))
ensure_dir(ax_iso_dst)
ensure_dir(sax_iso_dst)

target_spacing = (1.5,1.5,1.5)
# spawn multiple threads or processes
with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:# resample ax towards sax and vize versa
    
    for ax_file, sax_file in zip(ax_files, sax_files):
        
        reader1 = sitk.ImageFileReader()
        reader1.SetFileName(ax_file)
        ax_img = reader1.Execute()
        
        reader2 = sitk.ImageFileReader()
        reader2.SetFileName(sax_file)
        sax_img = reader2.Execute()
        
        # make a pseudo isotrop volume from both input images
        if 'img' in os.path.basename(ax_file):
            futures_ax_iso.add(executor.submit(transform_to_isotrop_voxels, ax_img, sitk.sitkLinear,target_spacing, ax_file))
            futures_sax_iso.add(executor.submit(transform_to_isotrop_voxels, sax_img, sitk.sitkLinear,target_spacing, sax_file))
        else:
            futures_ax_iso.add(executor.submit(max_thres_resample2_iso_label_img, ax_img,50, target_spacing, ax_file, sitk.sitkLinear))
            futures_sax_iso.add(executor.submit(max_thres_resample2_iso_label_img, sax_img,50, target_spacing, sax_file, sitk.sitkLinear))

# this part is sequential, we need only one writer
writer = sitk.ImageFileWriter()
for future in as_completed(futures_ax_iso):
    try:
        res = future.result()
        iso_resampled, file_path = res
        writer.SetFileName(os.path.join(ax_iso_dst,os.path.basename(file_path)))
        writer.Execute(iso_resampled)

    except Exception as e:
        logging.error(
            'Exception {} in datagenerator with: image: {} or mask: {}'.format(str(e)))
        
for future in as_completed(futures_sax_iso):
    try:
        res = future.result()
        iso_resampled, file_path = res
        writer.SetFileName(os.path.join(sax_iso_dst,os.path.basename(file_path)))
        writer.Execute(iso_resampled)

    except Exception as e:
        logging.error(
            'Exception {} in datagenerator with: image: {} or mask: {}'.format(str(e)))

In [57]:
# resample 3D image / mask into a fake isotrop resolution, use multithreadding
import concurrent.futures
from concurrent.futures import as_completed

workers = 16
futures = set()
files = sorted(glob.glob(sax_src+'*.nrrd', recursive = True))[:2]
ensure_dir(dst)
# spawn multiple threads or processes
with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
    
    for file_path in files:
    
        reader1 = sitk.ImageFileReader()
        reader1.SetFileName(file_path)
        sitk_img = reader1.Execute()
    
        if 'img' in os.path.basename(file_path):
            futures.add(executor.submit(transform_to_isotrop_voxels, sitk_img, sitk.sitkLinear,(1.5,1.5,1.5), file_path))

        else:
            futures.add(executor.submit(max_thres_resample2_iso_label_img, sitk_img,50, file_path))

for i, future in enumerate(as_completed(futures)):
    try:
        res = future.result()
        iso_resampled, file_path = res
        writer = sitk.ImageFileWriter()
        writer.SetFileName(os.path.join(dst,os.path.basename(file_path)))
        writer.Execute(iso_resampled)

    except Exception as e:
        logging.error(
            'Exception {} in datagenerator with: image: {} or mask: {}'.format(str(e)))

In [125]:
#resample 3D Image into isotrop and write into dst
for file_path in sorted(glob.glob(sax_src+'*.nrrd', recursive = True)):
    file_path
    
    reader1 = sitk.ImageFileReader()
    reader1.SetFileName(file_path)
    sitk_img = reader1.Execute()
    
    if 'img' in os.path.basename(file_path):
        iso_resampled = transform_to_isotrop_voxels(sitk_img, interpolate=sitk.sitkLinear, spacing_=(1.5,1.5,1.5))
    else:
        iso_resampled = max_thres_resample2_iso_label_img(sitk_img, 80)
    
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(dst,os.path.basename(file_path)))
    writer.Execute(iso_resampled) 

'/mnt/ssd/data/gcn/ax_sax_from_flo/ax3d/0000-0HQQW4ZN_2007-05-23_ED_img.nrrd'

'/mnt/ssd/data/gcn/ax_sax_from_flo/ax3d/0000-0HQQW4ZN_2007-05-23_ED_msk.nrrd'

after closing
int64


'/mnt/ssd/data/gcn/ax_sax_from_flo/ax3d/0000-0HQQW4ZN_2007-05-23_ES_img.nrrd'

'/mnt/ssd/data/gcn/ax_sax_from_flo/ax3d/0000-0HQQW4ZN_2007-05-23_ES_msk.nrrd'

...

'/mnt/ssd/data/gcn/ax_sax_from_flo/ax3d/0000-Z0QR21CJ_2005-11-02_ED_msk.nrrd'

after closing
int64


'/mnt/ssd/data/gcn/ax_sax_from_flo/ax3d/0000-Z0QR21CJ_2005-11-02_ES_img.nrrd'

'/mnt/ssd/data/gcn/ax_sax_from_flo/ax3d/0000-Z0QR21CJ_2005-11-02_ES_msk.nrrd'

after closing
int64


# Manual tests from flo, not needed currently

In [20]:
ax_src = '/home/rflo/master/all_with_7_worst_regi/ax3d/'
sax_src = '/home/rflo/master/all_with_7_worst_regi/sax3d/'
dst = '/home/rflo/master/all_with_7_worst_regi/testground/'

In [21]:
ensure_dir(dst)

In [ ]:
ax_src = sorted(glob.glob('/home/rflo/master/all_with_7_worst_regi/ax3d/'+'*.nrrd', recursive = True))[0]
sax_src = sorted(glob.glob('/home/rflo/master/all_with_7_worst_regi/sax3d/'+'*.nrrd', recursive = True))[0]

In [22]:
ax_src = sorted(glob.glob('/home/rflo/master/all_with_7_worst_regi/ax3d_iso_linear/'+'*.nrrd', recursive = True))[0]
sax_src = sorted(glob.glob('/home/rflo/master/all_with_7_worst_regi/sax3d_iso_linear/'+'*.nrrd', recursive = True))[0]

In [ ]:
ax_src = sorted(glob.glob('/home/rflo/master/all_with_7_worst_regi/ax3d_iso_linear/'+'*.nrrd', recursive = True))[28]
sax_src = sorted(glob.glob('/home/rflo/master/all_with_7_worst_regi/sax3d_iso_linear/'+'*.nrrd', recursive = True))[28]

In [23]:
print(ax_src)
print(sax_src)

/home/rflo/master/all_with_7_worst_regi/ax3d_iso_linear/0000-0HQQW4ZN_2007-05-23_ED_img.nrrd
/home/rflo/master/all_with_7_worst_regi/sax3d_iso_linear/0000-0HQQW4ZN_2007-05-23_ED_img.nrrd


In [24]:
from scipy.ndimage import affine_transform
#testing single pictures
ax_file_path = ax_src
sax_file_path = sax_src

reader1 = sitk.ImageFileReader()
reader1.SetFileName(ax_file_path)
ax_img = reader1.Execute()

reader2 = sitk.ImageFileReader()
reader2.SetFileName(sax_file_path)
sax_img = reader2.Execute()

In [25]:
for shift in range(-20,1,2):    
    resampled = resample_direcion_origin_spacing_shift(ax_img, sax_img, shift = (0,0,shift), interpolate=sitk.sitkLinear)
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(dst,'shift'+str((0,0,shift))+'.nrrd'))
    writer.Execute(resampled) 

In [ ]:
for shift in range(-20,21,5):    
    resampled = resample_direcion_origin_spacing_shift(ax_img, sax_img, shift = (shift,0,0), interpolate=sitk.sitkLinear)
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(dst,'shift'+str((shift,0,0))+'.nrrd'))
    writer.Execute(resampled) 
    resampled = resample_direcion_origin_spacing_shift(ax_img, sax_img, shift = (0,shift,0), interpolate=sitk.sitkLinear)
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(dst,'shift'+str((0,shift,0))+'.nrrd'))
    writer.Execute(resampled) 
    resampled = resample_direcion_origin_spacing_shift(ax_img, sax_img, shift = (0,0,shift), interpolate=sitk.sitkLinear)
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(dst,'shift'+str((0,0,shift))+'.nrrd'))
    writer.Execute(resampled) 

In [ ]:
"""
sitk_img1 = ax_img
sitk_img2 = sax_ixmg
    
label1_img1 = get_single_label_img(sitk_img1,1)
label2_img1 = get_single_label_img(sitk_img1,2)
label3_img1 = get_single_label_img(sitk_img1,3)
resampled_label1 = resample_img(label1_img1, sitk_img2)
resampled_label2 = resample_img(label2_img1, sitk_img2)
resampled_label3 = resample_img(label3_img1, sitk_img2)
"""
size = ax_img.GetSize()
spacing = ax_img.GetSpacing()
size_new = [int(s*p) for s,p in zip(size,spacing)]
size_new = tuple(size_new)

In [ ]:
resampler = sitk.ResampleImageFilter()
resampler.SetSize(size_new)
resampler.SetOutputSpacing((1,1,1))
resampler.SetOutputOrigin(ax_img.GetOrigin())
#resampler.SetInterpolator(sitk.sitkNearestNeighbor)
new_img = resampler.Execute(ax_img)

#sitk_img, size=(12, 256, 256), spacing=(8, 1.25, 1.25), interpolate=sitk.sitkNearestNeighbor


ax_array = sitk.GetArrayFromImage(new_img)
#ax_img.SetDirection(sax_img.GetDirection())
direction = sax_img.GetDirection()
direction = np.reshape(direction,(3,3))


#r = R.from_matrix(direction)
new_array = affine_transform(ax_array,direction)
new_img = sitk.GetImageFromArray(new_array)
new_img.SetOrigin(ax_img.GetOrigin())
new_img.SetDirection(sax_img.GetDirection())   

#resampled = max_thres_resample_label_img(ax_img, sax_img,50)
#resampled = percentage_resample_label_img(ax_img, sax_img,80)

writer = sitk.ImageFileWriter()
writer.SetFileName(os.path.join(dst,os.path.basename('img_ax_new.nrrd')))
writer.Execute(new_img) 
#writer = sitk.ImageFileWriter()
#writer.SetFileName(os.path.join(dst,os.path.basename('sax_0.nrrd')))
#writer.Execute(sax_img) 

#writer = sitk.ImageFileWriter()
#writer.SetFileName(os.path.join(dst,os.path.basename(ax_file_path)))
#writer.Execute(resampled) 

In [ ]:
png_path = '/home/rflo/master/data_search/ax'

In [ ]:
png_path = '/home/rflo/master/data_search/ax_to_sax/'

In [ ]:
from IPython.display import Image
from ipywidgets import interact
path_to_png = png_path
@interact
def show_png(i = (0, len(sorted(glob.glob(os.path.join(path_to_png,'*.png'))))-1)):
    f_name = sorted(glob.glob(os.path.join(path_to_png,'*.png')))[i]
    print(f_name)
    return Image(filename=f_name)

In [ ]:
ax_src = '/home/rflo/master/all_with_7_worst_regi/ax3d/'
sax_src = '/home/rflo/master/all_with_7_worst_regi/sax3d/'
dst = '/home/rflo/master/all_with_7_worst_regi/testground/'

In [ ]:
ax_src = '/home/rflo/master/all_with_7_worst_regi/ax3d_iso_linear/'
sax_src = '/home/rflo/master/all_with_7_worst_regi/sax3d_iso_linear/'
dst = '/home/rflo/master/all_with_7_worst_regi/testground/'

In [ ]:
ensure_dir(dst)

In [ ]:
src = ax_src

In [ ]:
#copy all msk into dst

for file_path in sorted(glob.glob(src+'*.nrrd', recursive = True)):
    if 'msk' in os.path.basename(file_path):
        shutil.copyfile(file_path,os.path.join(dst,os.path.basename(file_path)))
        